# Candidate Extraction : Diseases

This notebook is meant for in-house demonstration of candidate extraction and featurization of tables. It assumes an input file in XHTML format, a strict form of HTML that coincides with XML structure, allowing for easy display (HTML) and safe tree traversal (XML).

In [ ]:
%load_ext autoreload
%autoreload 2

### Candidate Extraction

First, import the 'HTMLParser' class to read HTML tables

In [16]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/diseases/diseases.xhtml')

The "TableParser" class divides the html doc into cells, adding a 'cell_id' attribute to each cell for future traversal, and creating "Cell" objects that have attributes such as row number, column number, html tag, html attributes, and any tags/attributes on a cells ancestors in the table.

In [17]:
from snorkel.parser import TableParser, PassParser
table_parser = TableParser()
pass_parser = PassParser()

As usual, pass these to a Corpus object for digestion.

In [18]:
# from snorkel.parser import Corpus
# %time corpus = Corpus(html_parser, table_parser)

from snorkel.parser import CorpusParser
cp = CorpusParser(html_parser, table_parser)
%time corpus = cp.parse_corpus(name='Diseases Corpus')

CPU times: user 153 ms, sys: 10.4 ms, total: 163 ms
Wall time: 250 ms


In [25]:
doc = corpus.documents[0]
print doc
print doc.tables[0]
print doc.tables[0].cells[0]
print doc.tables[0].cells[0].phrases[0]

Document('0', Corpus (Diseases Corpus))
Table('0', 0)
Cell('0', 0, 0, 'Disease')
Phrase('0', 0, 0, 0, u'Disease')
0


Load the good 'ole disease dictionary for recognizing disease names.

In [26]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Loaded 507899 disease phrases!


Here we use a new CandidateSpace object, CellNgrams. It inherits from Ngrams, and ensures that the Table context object is broken up into cells before being passed into the usual routine for pulling out Ngrams.

In [27]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
table_ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

ImportError: cannot import name TableNgrams

Passing the CandidateSpace, Matcher, and Context objects to a Candidates object, extraction is performed, and we see that a number of disease CellNgrams are returned.

In [ ]:
from snorkel.candidates import Candidates
%time candidates = Candidates(table_ngrams, disease_matcher, corpus.get_contexts())
for candy in candidates.get_candidates(): print candy

### Feature Generation

We can then generate features on our set of candidates, including *new and improved* table features!

In [ ]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer(candidates, corpus)

In [ ]:
featurizer.get_features_by_id(candidates.get_candidates()[0].id)

Ta-da! Next up: feeding these features into the learning machine.